In [23]:
import numpy as np
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn import *

/home/sahil/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/sahil/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/sahil/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  

In [28]:
train, test = pd.read_csv("train.csv"), pd.read_csv("test.csv")
IDs = test['id']
X_train, X_test = train['comment_text'], test['comment_text']
X_test.loc[X_test.isnull()] = " " # replace the 1 NaN value in test
Y_train = train[train.columns[2:]]

In [25]:
tfv = TfidfVectorizer(min_df=3, max_df=0.9, max_features=None, strip_accents='unicode',\
               analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,2), use_idf=1,\
               smooth_idf=1, sublinear_tf=1, stop_words='english')
print("tfidf-vectorizing train ...")
tfv.fit(X_train)
X_train = tfv.transform(X_train)
print("tfidf-vectorizing test ...")
X_test = tfv.transform(X_test)

tfidf-vectorizing train ...
tfidf-vectorizing test ...


In [29]:
tfidf = feature_extraction.text.TfidfVectorizer(stop_words='english', max_features=800000)
X_train = tfidf.fit_transform(X_train)

In [30]:
tfidf = feature_extraction.text.TfidfVectorizer(stop_words='english', max_features=800000)
X_test = tfidf.fit_transform(X_test)

In [ ]:
Y_train.shape[1]

6

In [ ]:
for i in range(Y_train.shape[1]):
    feature = Y_train.columns[i]
    print("\n%s:" % feature)
    print("Baseline: %.2f" % (Y_train.iloc[:,i].sum() / Y_train.shape[0]))
    clf = ensemble.ExtraTreesClassifier(n_jobs=-1, random_state=3)
    clf.fit(X_train, Y_train.iloc[:,i])
    print(cross_val_score(clf, X_train, Y_train.iloc[:,i], cv=3, scoring='f1'))
    exec("pred_%s = pd.Series(clf.predict_proba(X_test).flatten()[1::2])" % feature)
    print(clf.predict_proba(X_test).flatten())


toxic:
Baseline: 0.10


In [ ]:
submission = pd.DataFrame({
             'id': IDs.astype('str'),
             'toxic': pred_toxic,
             'severe_toxic': pred_severe_toxic,
             'obscene': pred_obscene,
             'threat': pred_threat,
             'insult': pred_insult,
             'identity_hate': pred_identity_hate
             })

In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
submission.info()